In [1]:
import pickle
import dotenv

In [2]:
import tqdm
import time
import re

In [3]:
from mmw.parse import *

In [4]:
dotenv.load_dotenv()

True

In [5]:
import os
import openai

openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
response['choices'][0]['text']

'\n\nThis is indeed a test'

In [6]:
all_parsed_essays = parse_essays()

In [7]:
marked_essays, filtered_marked_essays, all_questions = parse_annotations()

In [8]:
len(all_parsed_essays), len(filtered_marked_essays)

(77, 76)

In [9]:
#model = "text-davinci-003"
model = "ada:ft-mark-my-words-2023-05-01-11-44-26" #ada

In [10]:
LIMIT_ESSAYS = 1

In [11]:
all_answers = []
for i,essay_array in tqdm.tqdm(enumerate(all_parsed_essays[:LIMIT_ESSAYS])):
    
    essay_id = int(re.findall(r'\d+', essay_array[0])[-1])
    cur_row = filtered_marked_essays[filtered_marked_essays['essay id'] == essay_id]
    if len(cur_row) != 1:
        continue

    for j,k in enumerate(all_questions):
        
        prompt = assemble_prompt(essay_array[3][:-1], #implicit newline
                                 str(k).strip())

        response = openai.Completion.create(model=model, prompt=prompt, temperature=0, max_tokens=3)
        all_answers.append((i, j, prompt, response, int(cur_row[k])))
        
        time.sleep(.5)

1it [00:09,  9.18s/it]


In [12]:
len(all_answers)

13

In [13]:
all_answers[0]

(0,
 0,
 'Given is a student essay and a question to the essay. Please rate the Question with a number between 0 and 3, where 0 is "not at all satisfied" and 3 is "very satisfied.".\n\nEssay: """\nThe Cage\nIt felt like I was walking forever, i didn\'t even know were i was i just knew i was somewhere. A creepy aliway to be exazaced. To be honest i would rather\nbe at school than this nightmare well not to brag but i was not scared in the slightited, anyway desided just to keep walking to see what was down this\naliway, just as i was almost there thunder shot right though the sky it was like someone had a shotgun shot a bullet right next to my ear jezz i was not\nprepared for that, where there is thunder there is rain and i was not gonna get my new guchi shoes wet so i bulted to the end of the crusty aliway,\ni saw... well i didn\'t see anything other than cages, well i guess it was just a dream cause i don\'t own\nguchi shoes. OK enough of me being broke I gotta get to school. Time for

In [14]:
save_results('data/evaluations/run_4', all_parsed_essays, all_questions, all_answers)

In [15]:
_,_,all_answers_loaded = load_results('data/evaluations/run_4')

In [16]:
[(a[0], a[1], a[3]['choices'][0]['text']) for a in all_answers_loaded]

[(0, 0, ' 0\nAnswer'),
 (0, 1, ' 0\n\n'),
 (0, 2, ' 0\n\n'),
 (0, 3, ' 1\nQuestion'),
 (0, 4, ' 1\nQuestion'),
 (0, 5, ' 0\nQuestion'),
 (0, 6, ' 0\nQuestion'),
 (0, 7, ' 0\n\n'),
 (0, 8, ' 0\n\n'),
 (0, 9, ' 0\n\n'),
 (0, 10, ' 0\nQuestion'),
 (0, 11, ' 0\n\n'),
 (0, 12, ' 0\nAnswer')]

In [17]:
[(a[0],a[1],a[3]['choices'][0]['text']) for a in all_answers]

[(0, 0, ' 0\nAnswer'),
 (0, 1, ' 0\n\n'),
 (0, 2, ' 0\n\n'),
 (0, 3, ' 1\nQuestion'),
 (0, 4, ' 1\nQuestion'),
 (0, 5, ' 0\nQuestion'),
 (0, 6, ' 0\nQuestion'),
 (0, 7, ' 0\n\n'),
 (0, 8, ' 0\n\n'),
 (0, 9, ' 0\n\n'),
 (0, 10, ' 0\nQuestion'),
 (0, 11, ' 0\n\n'),
 (0, 12, ' 0\nAnswer')]